In [ ]:
!pip install qiskit qiskit_aer



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 91.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 94.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 79.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 4.1 MB/s eta 0:00:00


In [ ]:
import numpy as np
from qiskit_aer import AerSimulator
from qiskit import QuantumCircuit, transpile

In [ ]:
def dj_oracle(case, n):
    oracle_qc = QuantumCircuit(n+1)
    out = n  # output qubit index

    if case == 'balanced':
        a = np.random.randint(2, size=n)
        while not np.any(a):
            a = np.random.randint(2, size=n)
        for i, bit in enumerate(a):
            if bit == 1:
                oracle_qc.cx(i, out)

    if case == 'constant':
        if np.random.randint(2) == 1:
            oracle_qc.x(out)

    oracle_gate = oracle_qc.to_gate()
    oracle_gate.name = 'Oracle'
    return oracle_gate


In [7]:
def dj_algorithm(index, case):
  #decide for constants to output the same 0 or 1,
  #for balance output 0 or 1 excatly half rtimes
  qc = QuantumCircuit(index+1, index)
  out = index

  #x gate to mak it ket 1
  qc.x(out)

  #give it superpos
  qc.h(range(index+1))

  #query the magic oracle
  oracle = dj_oracle(case, index)
  qc.append(oracle, range(index+1))

  #undo superpos
  qc.h(range(index))
  qc.measure(range(index), range(index))
  return qc

In [22]:
# Define n first
n = 4
backend = AerSimulator()

# Now call the function
dj_circuit_constant = dj_algorithm(n, 'constant')
compiled = transpile(dj_circuit_constant, backend)
result = backend.run(compiled, shots=1024).result()
counts_constant = result.get_counts()

print("Constant oracle results:", counts_constant)

Constant oracle results: {'0000': 1024}


In [19]:
n = 4
backend = AerSimulator()

dj_circuit_balanced = dj_algorithm(n, 'balanced')
compiled = transpile(dj_circuit_balanced, backend)
result = backend.run(compiled, shots=1024).result()
counts_balanced = result.get_counts()

print("Balanced oracle results:", counts_balanced)

Balanced oracle results: {'0010': 1024}


In [36]:
n = 4
backend = AerSimulator()

dj_circuit_balanced = dj_algorithm(n, 'balanced')
compiled = transpile(dj_circuit_balanced, backend)
result = backend.run(compiled, shots=1024).result()
counts_balanced = result.get_counts()

print("Balanced oracle results:", counts_balanced)


Balanced oracle results: {'0100': 1024}


In [37]:
backend = AerSimulator()
dj_circuit = dj_algorithm(n, 'balanced')

dj_circuit.draw()

┌───┐     ┌─────────┐┌───┐┌─┐         
q_0: ┤ H ├─────┤0        ├┤ H ├┤M├─────────
     ├───┤     │         │├───┤└╥┘┌─┐      
q_1: ┤ H ├─────┤1        ├┤ H ├─╫─┤M├──────
     ├───┤     │         │├───┤ ║ └╥┘┌─┐   
q_2: ┤ H ├─────┤2 Oracle ├┤ H ├─╫──╫─┤M├───
     ├───┤     │         │├───┤ ║  ║ └╥┘┌─┐
q_3: ┤ H ├─────┤3        ├┤ H ├─╫──╫──╫─┤M├
     ├───┤┌───┐│         │└───┘ ║  ║  ║ └╥┘
q_4: ┤ X ├┤ H ├┤4        ├──────╫──╫──╫──╫─
     └───┘└───┘└─────────┘      ║  ║  ║  ║ 
c: 4/═══════════════════════════╩══╩══╩══╩═
                                0  1  2  3

In [38]:
from qiskit import transpile
from qiskit_aer import AerSimulator

backend = AerSimulator()

dj_circuit = dj_algorithm(n, 'balanced')

# Measure the first n qubits and draw the circuit
# dj_circuit.measure(range(n), range(n))
dj_circuit.draw()

┌───┐     ┌─────────┐┌───┐┌─┐         
q_0: ┤ H ├─────┤0        ├┤ H ├┤M├─────────
     ├───┤     │         │├───┤└╥┘┌─┐      
q_1: ┤ H ├─────┤1        ├┤ H ├─╫─┤M├──────
     ├───┤     │         │├───┤ ║ └╥┘┌─┐   
q_2: ┤ H ├─────┤2 Oracle ├┤ H ├─╫──╫─┤M├───
     ├───┤     │         │├───┤ ║  ║ └╥┘┌─┐
q_3: ┤ H ├─────┤3        ├┤ H ├─╫──╫──╫─┤M├
     ├───┤┌───┐│         │└───┘ ║  ║  ║ └╥┘
q_4: ┤ X ├┤ H ├┤4        ├──────╫──╫──╫──╫─
     └───┘└───┘└─────────┘      ║  ║  ║  ║ 
c: 4/═══════════════════════════╩══╩══╩══╩═
                                0  1  2  3

In [39]:
backend = AerSimulator()
compiled = transpile(dj_circuit, backend)

result = backend.run(compiled, shots=1024).result()
counts = result.get_counts()

print(counts)

{'0111': 1024}


In [41]:
# Analyze results
print("\n=== Deutsch-Jozsa Algorithm Results ===")
print(f"Constant oracle: {counts_constant}")
print(f"Balanced oracle: {counts_balanced}")

# Interpretation
constant_result = list(counts_constant.keys())[0]
balanced_result = list(counts_balanced.keys())[0]

print("\n=== Interpretation ===")
if constant_result == '0' * n:
    print("Constant oracle: All zeros detected → Function is CONSTANT")
else:
    print("Constant oracle: Non-zero detected → Function is BALANCED")

if balanced_result == '0' * n:
    print("Balanced oracle: All zeros detected → Function is CONSTANT")
else:
    print("Balanced oracle: Non-zero detected → Function is BALANCED")


=== Deutsch-Jozsa Algorithm Results ===
Constant oracle: {'0000': 1024}
Balanced oracle: {'0100': 1024}

=== Interpretation ===
Constant oracle: All zeros detected → Function is CONSTANT
Balanced oracle: Non-zero detected → Function is BALANCED


In [42]:
# Draw constant oracle circuit (text-based - always works)
print("=== Constant Oracle Circuit ===")
print(dj_circuit_constant.draw('text'))

print("\n" + "="*60 + "\n")

# Draw balanced oracle circuit
print("=== Balanced Oracle Circuit ===")
print(dj_circuit_balanced.draw('text'))
print("\n" + "="*60)
print("=== DEUTSCH-JOZSA ALGORITHM SUMMARY ===")
print("="*60)
print(f"\nProblem size: n = {n} qubits")
print(f"\nClassical approach:")
print(f"  - Worst case queries needed: 2^(n-1) + 1 = {2**(n-1) + 1}")
print(f"\nQuantum approach:")
print(f"  - Queries needed: 1")
print(f"  - Speedup: EXPONENTIAL!")
print(f"\nResults:")
print(f"  - Constant oracle returned: {list(counts_constant.keys())[0]}")
print(f"  - Balanced oracle returned: {list(counts_balanced.keys())[0]}")
print(f"\nConclusion:")
print(f"  The Deutsch-Jozsa algorithm successfully distinguished")
print(f"  between constant and balanced functions with a single")
print("="*60)

=== Constant Oracle Circuit ===
     ┌───┐     ┌─────────┐┌───┐┌─┐         
q_0: ┤ H ├─────┤0        ├┤ H ├┤M├─────────
     ├───┤     │         │├───┤└╥┘┌─┐      
q_1: ┤ H ├─────┤1        ├┤ H ├─╫─┤M├──────
     ├───┤     │         │├───┤ ║ └╥┘┌─┐   
q_2: ┤ H ├─────┤2 Oracle ├┤ H ├─╫──╫─┤M├───
     ├───┤     │         │├───┤ ║  ║ └╥┘┌─┐
q_3: ┤ H ├─────┤3        ├┤ H ├─╫──╫──╫─┤M├
     ├───┤┌───┐│         │└───┘ ║  ║  ║ └╥┘
q_4: ┤ X ├┤ H ├┤4        ├──────╫──╫──╫──╫─
     └───┘└───┘└─────────┘      ║  ║  ║  ║ 
c: 4/═══════════════════════════╩══╩══╩══╩═
                                0  1  2  3 


=== Balanced Oracle Circuit ===
     ┌───┐     ┌─────────┐┌───┐┌─┐         
q_0: ┤ H ├─────┤0        ├┤ H ├┤M├─────────
     ├───┤     │         │├───┤└╥┘┌─┐      
q_1: ┤ H ├─────┤1        ├┤ H ├─╫─┤M├──────
     ├───┤     │         │├───┤ ║ └╥┘┌─┐   
q_2: ┤ H ├─────┤2 Oracle ├┤ H ├─╫──╫─┤M├───
     ├───┤     │         │├───┤ ║  ║ └╥┘┌─┐
q_3: ┤ H ├─────┤3        ├┤ H ├─╫──╫──╫─┤M├
     ├───┤